# OCR cleanliness evaluation

In [1]:
import pandas as pd
import numpy as np
import spacy
import string
import json

In [5]:
df = pd.read_parquet('../data/processed/RZ_processed.parquet')

In [5]:
df.head()

,date,year,month,day,pub,head,full_text,href,text_len,w_count
0,1802-01-01,1802,1,1,Rigasche Zeitung,"St. Petersburg, den 20. Decemb.","St. Petersburg, den 20. Decemb.\n\n\tAuf Aller...",https://proc.dom.lndb.lv/file.axd?id=3377815&a...,2067,260
1,1802-01-01,1802,1,1,Rigasche Zeitung,"Paris, den 18ten December.","Paris, den 18ten December.\n\n\tDer Oberconsu!...",https://proc.dom.lndb.lv/file.axd?id=3377818&a...,3664,534
2,1802-01-01,1802,1,1,Rigasche Zeitung,"Haag, den 22sten December.","Haag, den 22sten December.\n\n\tIn kurzem erwa...",https://proc.dom.lndb.lv/file.axd?id=3377819&a...,967,135
3,1802-01-01,1802,1,1,Rigasche Zeitung,"Zürich, den 16ten December.","Zürich, den 16ten December.\n\n\tDie durch die...",https://proc.dom.lndb.lv/file.axd?id=3377822&a...,482,65
4,1802-01-01,1802,1,1,Rigasche Zeitung,"Wien, den 19ten December.","Wien, den 19ten December.\n\n\tDie Malcheser-N...",https://proc.dom.lndb.lv/file.axd?id=3377823&a...,1154,170


## Annotation

In [237]:
def create_ocr_gold_label_set(df, size):
    
    sample = df.sample(size)
    indices = list(sample.index)
    
    training_set_list = {}
    
    counter = 1
        
    for i in indices:
        print('--------', counter, '--------', '\n')
        full_text = sample.loc[i]['full_text']
        print(full_text)
        
        text_params = get_text_parameters(full_text)
        
        user_choice = ''
        while user_choice not in [0,1]:
            user_choice = int(input('Is this text readable? [0/1]'))
            
        training_set_list[i] = user_choice
        
        counter += 1
        print('\n\n\n\n\n')
            
    
    return training_set_list

In [238]:
third_gold_label_set = create_ocr_gold_label_set(df, 100)

-------- 1 -------- 

Meteorologische Beobachtungen in Riga.

	Morgens i t t a g s b e n d s
Dat.
Barom. i Therm. W. H.
Barom.
I Therm. I W. ! H.
Barom. I Therm. W. H.
10. Zuli 11. — 12. — 13. — 14. — 15. — 16. —
28" Ü'",3+12°,5 W. 27"l 1"',5+13° ; W. 28" 0"',3 +13°, 5 NW. 27"ll"',9 + 13°,5|9Ш. 27"1Г",2 + 13°,5 NW. 27"ll"',7-f 13° N. 27"ll'"f9;-j-ll0 W. bw. bw. bd. bw.
28" 0"',4j + 17°,5 SO. j bw. 27"li'",9; + i6° NW. bw. 28" 0'".5-i-15« NW.i bw. 27"11"',9!4.15»,5 NW.j h. I27"ll"',2| + 17° NW. h. 27"ll"',9+16° NW.j bw. [27"11'",6|+16° W. ! bw. 27"11'",7, + 12°,5|Ш. Rg. 27"10"',9| + 13° NW. bro. 28" 0'",3+l3&lt;&gt; NW. bw. 27"11'",8 +13&lt;&gt;,5! W. Rg. 27"I1"',2-j-12»,5 NW. bw. 27"ll"',3-j-13",5 NW. h. 27"w'",2!4.12«,5 SW.! bw.
bw. bw. h.

	Dat.

	Barom.

	10. Zuli

	28" Ü'",3+12°,5 W. 27"l 1"',5+13° ; W. 28" 0"',3 +13°, 5 NW. 27"ll"',9 + 13°,5|9Ш. 27"1Г",2 + 13°,5 NW. 27"ll"',7-f 13° N. 27"ll'"f9;-j-ll0 W.

	11. —
12. —

	13. —
14. —
15. —
16. —

	Ist zu drucken erlaubt. Im Namen de

Is this text readable? [0/1]1






-------- 4 -------- 

Mexiko. Aus Philadelphia, 10. September, wird der ' „Times" über das Verhallen der Vereinigten Staaten zu Mexiko geschrieben: So hoch auch gegenwärtig die Wogen der Politik in den Vereinigten Staaten gehen, so viel Aufmerksamkeit erübrigt dem Amerikanischen Volke immer, um dem. was in Mexiko vorgeht, mit gespanntem Interesse zu fc-laen Dampfer-Linien verbinden die bedeutendsten AmerMniichen Häfen mit i/nen Merikos, und die Canäle der Mittheilungen sind beständig offen, so daß die Amerikaner von den Vorgängen jenseits des Rio Grande fast eben so gut unterrichtet sind, als die Mexikaner selbst. Die Sympathien der Nation sind gegen Maximilian, und waS die Negierung betrifft, so ist sie aus ihrer ursprünglich neutralen Haltung so weit herausgetreten, daß nicht viel mehr an der ausgesprochenen Allianz mit seinen Gegnern fehlt, da sie dieselben in jeder Weise ausschließlich der offenen Trnppcusendungen unterstützt. Sollte der Kaiser v

Is this text readable? [0/1]1






-------- 9 -------- 

Anekdote.

	Ein Matrose auf einem Kriegsschiffe vor Plymonth bat den Kapitain um Erlaubniß, a»S Land gehen zu dürfen. Aber er blieb lange über seine Zeit auS, worüber er bei der Zurückkehr hart angelassen wurde. Er antwortete: es thäte ihm sehr leid; aber er hatte ausdrücklich eine Dilly (eine Art von Chaise, die bei Plymouth gewöhnlich ist) gemiethet, um desto schneller zurückzukommen; indessen hätte der Kutscher ihm auf eine halbe Guinee kein Geld herausgeben können, daher sei er genöthigt gewesen, zwischen Plymomh und Dock auf- und abzufahreu, bis die halbe Guinee verfahren gewesen. UnaJücklicherweise für den armen Jack traf eS sich, daß, als die halbe Guinee verfahren war, er sich gerade auf dem Flecke befand, von wo er aufbrach, und daß er eben so weit zu gehen hatte , als »b er ganz und gar nicht gefahren wäre.
Is this text readable? [0/1]1






-------- 10 -------- 

— (D i e neue lutheriscbe Pauls-Kirche.) Am Tilde der 

Is this text readable? [0/1]1






-------- 12 -------- 

Mayland, den 6ten Januar.

	Daß das Erdbeben am 25ften Dezember, früh um 2 Uhr, nicht unbedeutend gewefen sey, erhellet daraus, daß in der Stadt Reggio allein 17 Kamine einstürzten- In vielen Dörfern und Städten hörte man die Glocken anschlagen.
Is this text readable? [0/1]1






-------- 13 -------- 

*— Die neuesten Meldungen aus Afghanistan lassen kaum einen Zweifel darüber, daß das Blutbad in Kabul nicht lediglich von einem «uf eigene Faust revoltirenden afghanischen Truppentheile ausgegangen ist, sondern der Beginn in der Ausführung eines lange vorbereiteten verrätherischen Planes gegen England war, und zwar eines Planes, der von keinem geringeren aIS dem Emir Jatub Ehan selbst betrieben worden ist. Unter diesen Umständen dürste die den Engländern erwachsene Aufgabe die denkbar schwierigste sein, und der beginnende Krieg, von den Afghanen aIS heiliger Krieg gegen England geführt, kann für letzteres viel schwieriger und bl

Is this text readable? [0/1]1






-------- 20 -------- 

Frankreich.

	Paris, 19. October. (St.-A.) Ja der Münze wird wieder eine neue Medaille geprägt, nämlich die sür Mitglieder des Corps legis- { ktif, das demnächst zusammentreten soll. Bekanntlich wird sür jede Legislaturperiode eine desondere Medaille geprägt; dieselbe zeigt auf der einen Seite das Brustbild Louis NapoleonS, auf der anderen Seite den Namen des Volksvertreters, der fie erhalten soll, von einem Eichenkranze umgeben. Diese Medaille ist von Silber und verbürgt dem Träger derselben nicht nur die persönliche Unantastbarkeit allen Gerichten gegenüber, sondern auch den ungehinderten Zutritt zu allen öffentlichen Ereignissen zc.
Paris, 20. October. (Tel. Bur.) Die Bank von Frankreich hat ihr DiSconto von 6V2 aus 7/2 pCt. erhöht. Die 3proc. war bei Abgang der Depesche auf 66,50 gewichen.
Is this text readable? [0/1]1






-------- 21 -------- 

Inländische Nachrichten.


	Riga, 24. Mai. Zufolge Resolution Eines Erlauchte

...







-------- 81 -------- 

Frankreich.

	laa Pari«, 28. (16.) Juli. Die in den letzten
Tagen verbreitete Nachricht, dahin lautend, daß zwischen dem UntenichtSminister luleS Ferry und dem Referenten der SenatScommisfion, Jules Simon, bezüglich deS famosen Jesuiten. Paragraphen ein Kompromiß abgeschlossen worden sei, wird jetzt von Seiten des Ministers auf daS Entschiedenste dementirr, wenngleich andererseit« bestimmt versichert wird, daß allerdings ein Kompromißvorschlag von Herrn Jules Simon ausgearbeitet worden und bereits dem Präsidenten der Republik (mit dem Herr Simon persönlich sehr liirt) unterbreitet ist, damit er denselben seinem IlitterrichtSminister zur Annahme empfiehlt. AlleS daS wird aber erst im Herbste ausgetragen werden, da gar nicht mehr daran zu denken ist, daß der Senat vor Beginn der parlamentarischen Ferien zur Berathung deS UnterrichtSgesetzeS gelangt.
I Die Commission ist erst bis zum Artikel 3 des
 gekommen, hat über ein halbes Dutzend R von Sontraprojecten Be

Is this text readable? [0/1]1






-------- 84 -------- 

Todes- Anzeige.
Allen theilnehmenden Verwandten und Freunden zeige ich tief gebeugt an, daß meine innig geliebte Gattin Catharinavon Paul, geborene von Uschakow, nach Gottes unerforschlichem Rathschluß am S. April d. 1., mittags 13 Uhr, nach nicht voll Rl-monatlicher glücklicher Ehe mir durch den Tod entrissen worden ist, und verbinde hiemit die Bitte, sich Dienstag, am 8. April, vormittags 91/ s Iths, zu ihrer Bestattung, welche von der Alexander – Newsky – Kirche aus vollzogen werden soll, gütigst einfinden zu wollen.
F. v. Paul.

	Bekanntmachungen.

	Vom Livländischen Landraths -Collegio werden Diejenigen vom Livländischen Adel, welche in die Eivil-Jngenieurschule in St. Petersbürg als Zöglinge der Livländischen Ritterschaft einzutreten wünschen, ausgefordert, sogleich beim Laudraths-Collegio sich *u melden, und die in dem Patente Einer Livländischen Gouvernements-Regierung vom 22. September 1832 № 100 vorgeschriebenen Attes

Is this text readable? [0/1]1






-------- 89 -------- 

— Eine große Verwirrung herrscht in Ungarn seit Jahren auf dem Gebiete des Eherechts. Dem Lande fehlt ein einschlägiges Gesetz und es gelten daselbst nebeneinander die Eherechte der verschiedenen christlichen Confessionen und außerdem noch für die Juden ein Hofkanzleidecret. Diese Verwirrung ist durch die Einführung der Nothcivilehe in Oesterreich, welche die Ehe zwischen Christen und Confessionslosen für zulässig erklärte, noch vermehrt worden. Die ungarischen Behörden erkannten nämlich den in Oesterreich von ungarischen Staatsbürgern abgeschlossenen Civilehen keine Giltigkeit zu, und ebenso erklärten sie, daß der Uebertritt eines ungarischen Staatsbürgers christlicher Religion zum Judenthume ein rechtlich unwirksamer und daher null und nichtiger Act sei, so daß eine Ehe, die ein zum Judenthnme Uebergetretener mit einem jüdischen Ehetheile eingeht, gleichfalls der Rechtsgiltigkeit entbehrt. Aus diesen Verhältnissen ergaben sic

Is this text readable? [0/1]1






-------- 94 -------- 

Warschau, den 29. April.

	Der Admiiuslran'Msrach des Königreichs Polen hat Kit Rücksicht darauf, daH, k« Folge der Fruchtbarkeit des letzten Jahres und bei dem Mangel an Absatz von Gettaide im Auslande, der Preis desselben so nieHrig geworden ist, daß er den Producenten die Auslage» nicht einbringt, auf Vorstellung des präsidirenden General-Dircctors der Regierungs-Lommisswn der Finanzen, um dem Ackerbau aufzuhelfen, die Verordnung erlassen, daß ' die Polnische Bank den Landeseinrvohnern, gegen Depokiruna von Walzen und Roggen unter ihrem Verschluß ln den Magazinen und anderen zu diesem Zweck auszuwählenden Niederlagen, die gut verwahrt und in der Feuerkasse versichert seyn müssen, Vorschüsse bewilligen soll« Mit diesen Vorschüssen kann nöthigenfalls bis zu 3 Mill. Gulden gegangen.werden; sollte sich aber auch diese Quote noch nicht als hinreichend erweisen, so hat die Bank die Vollmacht, zur weiteren Ausdehnung derselben nach

Is this text readable? [0/1]1








In [241]:
### save gold label data

with open('../pipeline/02_ocr/readability_gold_label.json', 'w', encoding='utf8') as f:
    contents = json.dumps(third_gold_label)
    f.write(contents)

## Training

In [9]:
nlp = spacy.load('de_core_news_md')

In [7]:
common_punctuation = ['.', ',', '?', '!', ';', '-', ':']
common_symbols = set(string.ascii_lowercase).union(string.ascii_uppercase).union(common_punctuation)
POS_tags = ['PRON', 'ADV', 'AUX', 'NUM', 'VERB', 'X', 'ADJ', 'SPACE', 'PUNCT']

def get_text_parameters(text):
    
    doc = nlp(text)
    doc_properties = {}
    
    # proportions of token POS in doc
    doc_pos = [token.pos_ for token in doc]
    for cat in POS_tags:
        doc_properties[cat] = doc_pos.count(cat)/len(doc)
    
    # proportions of weird symbols
    uncommon_symbols = 0
    for symbol in text:
        if symbol not in common_symbols:
            uncommon_symbols += 1
            
    # character count to token proportion
    doc_properties['char_to_word'] = len(text)/len(doc)
            
    doc_properties['uncommon_symb_proportion'] = uncommon_symbols/len(text)
    
        
    return doc_properties  

In [3]:
### open gold label data

with open('../pipeline/readability_gold_label.json', 'r', encoding='utf8') as f:
    contents = f.read()
    gold_label = json.loads(contents)

In [10]:
gold_label_data = []

for key in gold_label.keys():
    
    entry_text = df.iloc[int(key)].full_text
    text_params = get_text_parameters(entry_text)
    text_params['readable'] = gold_label[key]
    
    gold_label_data.append(text_params)

In [11]:
training_set = pd.DataFrame(gold_label_data, index=gold_label.keys())
training_set.head()

,PRON,ADV,AUX,NUM,VERB,X,ADJ,SPACE,PUNCT,char_to_word,uncommon_symb_proportion,readable
146680,0.038113,0.079855,0.038113,0.001815,0.085299,0.003630,0.070780,0.012704,0.134301,5.950998,0.166209,1
235187,0.036545,0.069767,0.039867,0.003322,0.083056,0.033223,0.066445,0.003322,0.132890,6.073090,0.173961,1
5748,0.039261,0.073903,0.013857,0.013857,0.057737,0.000000,0.108545,0.016166,0.120092,5.755196,0.180177,1
6755,0.026549,0.053097,0.026549,0.026549,0.053097,0.017699,0.061947,0.026549,0.106195,5.876106,0.162651,1
51850,0.050239,0.083732,0.033493,0.028708,0.071770,0.014354,0.069378,0.007177,0.169856,5.519139,0.188990,1


### Multiple regression

In [13]:
from sklearn.model_selection import train_test_split

x = training_set.drop(['readable'], axis=1).values
y = training_set['readable'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [14]:
from sklearn.linear_model import LinearRegression

ml = LinearRegression()
ml.fit(x_train, y_train)

y_pred = ml.predict(x_test)

from sklearn.metrics import r2_score

r2_score(y_test, y_pred)

0.6030962743800508

### Support Vector Machine

In [15]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(x_train, y_train)
svc_y_pred = svc.predict(x_test)

In [16]:
svc.score(x_test, y_test)

0.8666666666666667

In [18]:
import pickle

with open('../data/models/ocr_quality_model_150921/ocr_quality_model.pkl', 'wb') as f:
    pickle.dump(svc, f)

## Random predictions

In [433]:
def predict_ocr(index, model):
    
    text = df.iloc[index].full_text
    
    params = get_text_parameters(text).values()
    
    x = np.array(list(params))
    x = x.reshape(1,-1)

    print(text, '\n\n')
    return model.predict(x)

In [470]:
def predict_random(model):
    
    num = np.random.randint(0, len(df))
    print(predict_ocr(num, model=model))

In [487]:
predict_random(model=svc)

London, den 27. August.

	Heute Nackmittag um 2 Uhr verließ Ihre Majestat den Palast und begab sich nach dem Parlamente, dieses zu vertagen. Die Konigin wurde dort von dem Lord-Kanzler und den vornehmsten Staats – Beamten empfange». I. Maj. sah sehr wohl aus und hielt die Rede sehr vernehmlich und mit bcsonderm Nach-

	druck. Sie begann damit, daß sie dem Lande zu dem Abschluß de« Desinitiv-Vertrag« zwischen Holland und Belgien Glück wünschte und sagte, daß eben dieser Geist der Eintracht unter den fünf Mächten, welcher diese Verständigung zu Wege gebracht, auch Hinsichtlich der orientalischen Angelegenheiten bestehe, wobei sie hinzufügte, „daß die Mächte eben so entschlossen wären, die Unabhängigkeit und Unverletzlichteit de« Ottomanischen Reich« aufrecht zu erhalten", eineGe sinnung, von der sie erwarte, daß sie eine befriedigende Ausgleichung der Begebenheiten in jener Gegend herbeiführen «erde. Hierauf kommt der Glückwunsch wegen der glücklichen Ermittelung Großbritannien« zwischen

In [27]:
training_set.loc['146680'].values

array([3.81125227e-02, 7.98548094e-02, 3.81125227e-02, 1.81488203e-03,
       8.52994555e-02, 3.62976407e-03, 7.07803993e-02, 1.27041742e-02,
       1.34301270e-01, 5.95099819e+00, 1.66209210e-01, 1.00000000e+00])

In [30]:
dictvalues = get_text_parameters(df.loc[700, 'full_text']).values()

In [32]:
x = np.array(list(dictvalues))
x = x.reshape(1,-1)

In [33]:
x

array([[0.00884956, 0.04424779, 0.03539823, 0.00884956, 0.07079646,
        0.00884956, 0.07079646, 0.02654867, 0.10619469, 5.86725664,
        0.173454  ]])

In [34]:
svc.predict(x)

array([1], dtype=int64)

In [35]:
df.loc[700, 'full_text']

'Die noch zu Neapel befindlichen Russischen Grenadiers haben am astend. M. beyPortici unter dem GeneralßoroSdin, in Gegenwart des Königs und der Königin von Neapel, ein schönes Manövre gehalten, worauf der König die Grenadiers z Tage bewirthen ließ.\nAuf der Insel Elba sind von den Franzosen mehrere Matrofen gepreßt und zur Bemannung von Kriegsschiffen nach Toulon gebracht worden.\nUm die inländischen Manufacturen in Aufnahme zu bringen, hat der heil. Vater verordnet, daß vom lsten September an alle ausländische Baumwollen- und Wöllenwaaren im Kirchenstaat 18 Procent und in Rom besonders noch i Procent Einfuhrzoll entrichten follen.\n\n\tMayland, den 31. Julii.'